# Introduction to NeoRV32OnPynq


## Aims

* Introduce the compiler magic provided by NeoRV32OnPynq, and how to use it. 
* Introduce the NeoRV32OnPynq overlay GUI

# Introducing the Compiler Magic


This section will introduce the compiler magic provided by NeoRV32OnPynq, riscvc.
This magic enables C programs to be written in any Jupiter Notebook cell, and then compiled simply by running said cell
Before this magic can be used it must be loaded, which is done by the line below

In [3]:
%load_ext NeoRV32OnPynq

The NeoRV32OnPynq extension is already loaded. To reload it, use:
  %reload_ext NeoRV32OnPynq


The compiler magic is called using <code> %%riscvc <i>progam_name</i> </code>. 
The compiled program can be found in <code> programs/<i>progam_name</i>/<i>progam_name</i>.bin </code>, where <code>programs</code> is a folder in the same location as the notebook in which the compiler magic was run. 

The `programs` folder, and its children (each named for the program they contain), serve as a way to  prevent different programs from inferring with each other. Each child folder contains not only the compiled program (.bin), but also any intermedate files (.c, .o), and some files useful for debugging (.asm, .coe).

<div class="alert alert-box alert-info">
There is no need to worry about creating a <code>programs</code> folder, or any of its children, as the compiler magic handles this as needed.
</div>

<div class="alert alert-box alert-info">
The compiler magic is a cell magic, meaning that it needs to be the first statement in a cell, and the rest of the cell's content will be interpreted by the magic.
</div>

## Compiler Magic Example

Below is an example of how to use the compiler magic.
Upon running the cell below a number of effects should be seen:
* If there wasn't already a `programs` folder, one will have been created
* If there wasn't already a `programs\flash_LEDs` folder, one will have been created
* A makefile was added to `programs\flash_LEDs`
* `programs\flash_LEDs\flash_LEDs.c`was (re)created and contains everything but the first line of the compiler magic cell 
* `programs\flash_LEDs\flash_LEDs.c.o` was (re)created
* `programs\flash_LEDs\flash_LEDs.bin` was (re)created
* `programs\flash_LEDs\flash_LEDs.asm` was (re)created
* `programs\flash_LEDs\flash_LEDs.coe` was (re)created


In [15]:
%%riscvc flash_LEDs

// Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define outsets for LED GPIO handler's regmap
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;
    
    // Set the LEDs in off state
    LEDs[value] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        LEDs[value] = ~LEDs[value];
    }

    return 0;
}



mkdir -p programs/flash_LEDs

cp /usr/local/lib/python3.6/dist-packages/NEORV32_on_PYNQ/makefile programs/flash_LEDs/makefile

make clean exe NEORV32_HOME=/usr/local/lib/python3.6/dist-packages/NEORV32_on_PYNQ/NEORV_lib LD_SCRIPT=/usr/local/lib/python3.6/dist-packages/NEORV32_on_PYNQ/NEORV32_on_PYNQ.ld
Memory utilization:
   text	   data	    bss	    dec	    hex	filename
    412	      0	      0	    412	    19c	main.elf
Executable (neorv32_exe.bin) size in bytes:
424

mv main.bin flash_LEDs.bin

mv main.asm flash_LEDs.asm

bin2coe -i flash_LEDs.bin -o flash_LEDs.coe -w 32

rm main.* neorv32_exe.bin



## Compiler Magic Example 2


A notebook is not restricted to compiling a single program, below is an example of the compiler magic being used to compile another program, the same set of effects should be seen upon running this cell.

In [17]:
%%riscvc LED_counter

// Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define outsets for LED GPIO handler's regmap
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        for(int32_t i=0; i < 16; i++)
        {
            LEDs[value] = i;
        }
    }

    return 0;
}

mkdir -p programs/LED_counter

cp /usr/local/lib/python3.6/dist-packages/NEORV32_on_PYNQ/makefile programs/LED_counter/makefile

make clean exe NEORV32_HOME=/usr/local/lib/python3.6/dist-packages/NEORV32_on_PYNQ/NEORV_lib LD_SCRIPT=/usr/local/lib/python3.6/dist-packages/NEORV32_on_PYNQ/NEORV32_on_PYNQ.ld
Memory utilization:
   text	   data	    bss	    dec	    hex	filename
    416	      0	      0	    416	    1a0	main.elf
Executable (neorv32_exe.bin) size in bytes:
428

mv main.bin LED_counter.bin

mv main.asm LED_counter.asm

bin2coe -i LED_counter.bin -o LED_counter.coe -w 32

rm main.* neorv32_exe.bin



# Introducing the NeoRV32OnPynq Overlay GUI



Now that we have some programs compiled for the NeoRV32OnPynq overlay, lets looks at how to interact with the overlay.
But before that we have to load the overlay onto the board, this is done by the code below.
It loads the NeoRV32OnPynq library, then called the overlay class within it which loads the overlay onto the board and returns a handle to overlay.

In [18]:
import NeoRV32OnPynq
ol = NeoRV32OnPynq.overlay()

The recommended way to interact with the overlay is via the GUI provided; this GUI consists of 3 main components:
* load_program, selection and loading of programs into the overlay's BRAM
* execution_control, controls when the overlay executes the program, enabling simple and conditional stepping through the program
* show_internals, display a range of signals internal to the overlay, to aid in debugging and/or tracking of the instructions though the overlay

Each of the parts will be covered in their own section.

## load_program GUI


The load program GUI is responsbile for enabling a user to select any of the programs found in `programs` and loading them into the overlays BRAM for execution. It is displayed using the function shown below

In [21]:
ol.load_program()

There are four points of interest on this GUI:-
* The refresh programs button, when pressed this button rescans`programs` and updates the GUI's list of available programs e.g. removing any  that have been deleted or adding newly compiled programs
* The program select dropdown, this dropdown lists all programs known to the GUI letting you select which you wish to load on to the overlay
* The load program button, when clicked this button will read the binary of the program selected using the program select dropdown and load it into the overlay BRAM and reset the pc to the start of the program
* The last loaded field, this field shows the name of the last program loaded by the GUI. This will normal be the program currently running on the board, assuming the board has remained connected and only the GUI has been used for loading programs.

## execution_control GUI

The execution_control GUI is responsible for controling the overlay's NERORV core's clock, this enables control over when the NERORV core can execute instructions. This control can vary from  running in single clock cycle steps to running the overlay until a specific position within the program is reached. The execution_control GUI is displayed using the code below

In [19]:
 ol.execution_control()

## show_internals GUI

display a range of signals internal to the overlay, to aid in debugging and/or tracking of the instructions though the overlay

In [22]:
ol.show_internals()